## Import the Required Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import contextily as ctx
import seaborn as sns
import geoplot as gplt
import geoplot.crs as gcrs
import folium
import matplotlib.ticker as ticker
import scipy.stats
import webbrowser
from scipy.stats import shapiro
import statsmodels.api as sm
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from sklearn.preprocessing import MinMaxScaler
import matplotlib
from IPython.display import display, HTML, Image
import json
from scipy import stats
%matplotlib inline
import dataframe_image as dfi
import mgwr
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import compare_surfaces, truncate_colormap
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Point
import pysal
from pysal.model import spreg
import textwrap
import matplotlib.colors as colors#
import statistics

## Import the Datasets

In [31]:
# PPFI version 2
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/PPFI (V2).csv"
PPFI_V2 = pd.read_csv(file_path)

# LSOA 2011 to LSOA 2021
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/LSOA (2011) to LSOA (2021).csv"
LSOA11_LSOA21 = pd.read_csv(file_path)

# Oxfordshire LSOAs 2011
file_path = "C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/Oxfordshire LSOAs.csv"
Ox_LSOA11 = pd.read_csv(file_path)

## Merge the Datasets

In [32]:
PPFI_V2_Ox_Reordered = PPFI_V2.merge(LSOA11_LSOA21, on='LSOA21CD', how='inner')

In [33]:
PPFI_V2_Ox_Reordered = PPFI_V2_Ox_Reordered.merge(Ox_LSOA11[['LSOA11CD']], on='LSOA11CD', how='inner')

## Create the quantiles for the combined dimension

In [34]:
PPFI_V2_Ox_Reordered['pp_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['combined'],
                                                 q=10,
                                                  labels=range(1,11))

## Create the quantiles for the separate PPFI dimensions

In [35]:
PPFI_V2_Ox_Reordered['SM_Prox_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_supermarket_proximity'],
                                             q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['SM_Acc_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_supermarket_accessibility'],
                                            q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['NonSM_Prox_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_nonsupermarket_proximity'],
                                            q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['fuelPov_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_fuel_poverty'],
                                             q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['SD_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_socio_demographic'],
                                            q=10, labels=range(1,11))
PPFI_V2_Ox_Reordered['FFF_dec_Ox'] = pd.qcut(PPFI_V2_Ox_Reordered['domain_food_for_families'],
                                             q=10, labels=range(1,11))

In [36]:
PPFI_V2_Ox_Reordered['E_Comm_dec_Ox'] = pd.cut(PPFI_V2_Ox_Reordered['domain_ecommerce_access'],bins=10, labels=range(1,11))

In [37]:
PPFI_V2_Ox_Reordered = PPFI_V2_Ox_Reordered.drop(['domain_supermarket_proximity',
       'domain_supermarket_accessibility', 'domain_ecommerce_access',
       'domain_socio_demographic', 'domain_nonsupermarket_proximity',
       'domain_food_for_families', 'domain_fuel_poverty', 'country',
       'country_denominator', 'combined',
       'pp_dec_domain_supermarket_proximity',
       'pp_dec_domain_supermarket_accessibility',
       'pp_dec_domain_ecommerce_access', 'pp_dec_domain_socio_demographic',
       'pp_dec_domain_nonsupermarket_proximity',
       'pp_dec_domain_food_for_families', 'pp_dec_domain_fuel_poverty',
       'pp_dec_combined',],axis=1)

In [38]:
PPFI_V2_Ox_Reordered = PPFI_V2_Ox_Reordered.rename(columns=
{
    'pp_dec_Ox':'PPFI Combined Decile',
    'SM_Prox_dec_Ox':'Supermarket Proximity',
    'SM_Acc_dec_Ox':'Supermarket Accessibility',
    'NonSM_Prox_dec_Ox':'Non-Supermarket Proximity',
    'fuelPov_dec_Ox': 'Fuel Poverty',
    'SD_dec_Ox':'Socio-demographic Barriers',
    'FFF_dec_Ox':'Food Support for Families',
    'E_Comm_dec_Ox':'E-Commerce Access'
})

In [43]:
PPFI_V2_Ox_Reordered.head()

,LSOA21CD,LSOA11CD,LSOA11NM,LSOA21NM,PPFI Combined Decile,Supermarket Proximity,Supermarket Accessibility,Non-Supermarket Proximity,Fuel Poverty,Socio-demographic Barriers,Food Support for Families,E-Commerce Access
0,E01028422,E01028422,Cherwell 008A,Cherwell 008A,4,3,3,1,6,6,7,8
1,E01028423,E01028423,Cherwell 008B,Cherwell 008B,8,3,2,1,9,10,7,8
2,E01028426,E01028426,Cherwell 007A,Cherwell 007A,6,9,6,2,9,6,4,6
3,E01028427,E01028427,Cherwell 007B,Cherwell 007B,5,10,8,4,3,3,2,9
4,E01028428,E01028428,Cherwell 007C,Cherwell 007C,8,9,7,2,10,7,5,6


In [48]:
PPFI_V2_Ox_Reordered['District'] = PPFI_V2_Ox_Reordered['LSOA21NM'].str.extract('^([^0-9]+)')
PPFI_V2_Ox_Reordered['District'] = PPFI_V2_Ox_Reordered['District'].str.strip()

## Export to an Excel file

In [50]:
PPFI_V2_Ox_Reordered.to_excel("C:/Users/medaamm/OneDrive - University of Leeds/Project 1 Final Paper/DSDP-Project-1/Datasets/PPFI V2 Ox Reordered.xlsx")